# **Strain Gage Calculations - v1**

- All the required input files should be in the same directory as this notebook
- Output files will be in the same directory as well.

--------
#### **Import Main Modules**

In [9]:
import pandas as pd
import numpy as np
from openpyxl import load_workbook

------

#### **Define Material Properties**

In [10]:
# Define the material properties and the strain gauge angles

E = 200e9  # Young's Modulus in Pascals (Pa)
v = 0.3  # Poisson's ratio

--------
#### **Functions for Converting the Measured Rosette Strains into Various Outputs** 

References for the formulas used: 

https://community.sw.siemens.com/s/article/rosette-strain-gauges

https://www.youtube.com/watch?v=9x-3H74a8YQ


In [11]:
def transform_strains_to_global(epsilon_A, epsilon_B, epsilon_C, angles):
    """
    Transform strains from the rosette's local coordinate system to the global coordinate system.

    Parameters:
    - epsilon_A (float): Measured normal strain from the first strain gauge in the rosette.
    - epsilon_B (float): Measured normal strain from the second strain gauge in the rosette.
    - epsilon_C (float): Measured normal strain from the third strain gauge in the rosette.
    - angles (list of float): The angles (in degrees) of the strain gauges relative to the global x-axis.

    Returns:
    - numpy.array: The transformed strains in the global coordinate system, containing
      the normal strains epsilon_x and epsilon_y, along with the engineering shear strain gamma_xy.
    """
    theta_A, theta_B, theta_C = np.radians(angles)
    T = np.array([
        [np.cos(theta_A)**2, np.sin(theta_A)**2, 1 * np.sin(theta_A) * np.cos(theta_A)],
        [np.cos(theta_B)**2, np.sin(theta_B)**2, 1 * np.sin(theta_B) * np.cos(theta_B)],
        [np.cos(theta_C)**2, np.sin(theta_C)**2, 1 * np.sin(theta_C) * np.cos(theta_C)]
    ])
    T_inv = np.linalg.inv(T)
    local_strains = np.array([epsilon_A, epsilon_B, epsilon_C])
    global_strains = T_inv @ local_strains
    return global_strains

def calculate_principal_strains(epsilon_x, epsilon_y, gamma_xy):
    """
    Calculate the principal strains using Mohr's circle relations.

    Parameters:
    - epsilon_x (float): Normal strain in the x-direction (global coordinate system).
    - epsilon_y (float): Normal strain in the y-direction (global coordinate system).
    - gamma_xy (float): Engineering shear strain in the global coordinate system.

    Returns:
    - numpy.array: An array containing the maximum (epsilon_1) and minimum (epsilon_2)
      principal strains. These values are critical for assessing the material's behavior
      under stress and for failure analysis.

    Note:
    - The shear strain input should be in engineering terms (total angular deformation),
      as this is the standard output from strain gauges.
    """
    C = (epsilon_x + epsilon_y) / 2
    R = np.sqrt(((epsilon_x - epsilon_y) / 2)**2 + (gamma_xy / 2)**2)
    epsilon_1 = C + R  # Maximum principal strain
    epsilon_2 = C - R  # Minimum principal strain
    return np.array([epsilon_1, epsilon_2])

def calculate_principal_stresses(principal_strains, E, v):
    """
    Calculate the principal stresses from the principal strains using material properties.

    Given the principal strains, this function applies Hooke's law in two dimensions to
    compute the principal stresses. The material's Young's modulus (E) and Poisson's ratio (v)
    are used to relate the strains to the stresses. This function assumes a linear elastic
    material behavior and plane stress conditions, which is a common scenario in thin structures
    where one dimension is significantly smaller than the other two.

    Parameters:
    - principal_strains (numpy.array): An array of the principal strains [epsilon_1, epsilon_2].
    - E (float): Young's Modulus of the material in Pascals (Pa).
    - v (float): Poisson's ratio of the material, dimensionless.

    Returns:
    - numpy.array: An array containing the principal stresses [sigma_1, sigma_2] in Pascals (Pa).
      These stresses are the maximum and minimum normal stresses that occur at the principal
      strain orientations, where the shear stress is zero.

    Note:
    - The principal strains should be provided in microstrains (με) for the calculation.
    - The function returns the principal stresses in Pascals (Pa), but they are manually
      converted to other MPa (MegaPascals) at the function return (by a /1e6 division).
    - The function holds true for isotropic materials where the stress-strain relationship 
      is governed by the isotropic form of Hooke's law.
    """
    #S = np.array([
    #    [1, v, 0],
    #    [v, 1, 0],
    #    [0, 0, (1-v)/2]
    #]) * E / (1 - v**2)
    S = np.array([
        [1, v],
        [v, 1]
    ]) * E / (1 - v**2)

    principal_stresses = S @ (principal_strains /1e6)
    return principal_stresses / 1e6  # Convert Pa to MPa

def calculate_principal_strain_orientation(epsilon_x, epsilon_y, gamma_xy):
    """
    Calculate the orientation of the principal strains from the original strain measurements using Mohr's circle analysis.
    Parameters:
    - epsilon_x (float): Normal strain in the x-direction.
    - epsilon_y (float): Normal strain in the y-direction.
    - gamma_xy (float): Engineering shear strain.
    Returns:
    - theta_p (float): Angle of the principal strain in degrees.
    """
    # Calculate the angle to the maximum principal strain
    theta_p_rad = 0.5 * np.arctan2(gamma_xy, epsilon_x - epsilon_y)
    theta_p = np.degrees(theta_p_rad)

    # Adjust the angle to ensure it's within the 0-180 degree range
    if theta_p < 0:
        theta_p += 180

    return theta_p

def calculate_biaxiality_ratio(S1, S2):
    """
    Calculate the biaxiality ratio from the principal stresses.

    The biaxiality ratio is the ratio of the second principal stress to the first principal stress.
    It is a dimensionless number that indicates the relation between the principal stresses. A
    biaxiality ratio of 1 indicates equal biaxial stress state, while a value of 0 indicates a uniaxial
    stress state. Negative values indicate that the principal stresses are of opposite signs.

    Parameters:
    - S1 (float or numpy.array): First principal stress (assumed to be the larger one in absolute value).
    - S2 (float or numpy.array): Second principal stress (assumed to be the smaller one in absolute value).

    Returns:
    - float or numpy.array: The biaxiality ratio, a dimensionless quantity indicating the biaxial state of stress.

    Note:
    - The function assumes that S1 and S2 are provided such that the absolute value of S1 is
      greater than or equal to the absolute value of S2. If this is not the case, the inputs
      should be swapped.
    """
    # Ensure that sigma_1 is the larger one in absolute terms
    sigma_1 = np.where(np.abs(S1) >= np.abs(S2), S1, S2)
    sigma_2 = np.where(np.abs(S1) >= np.abs(S2), S2, S1)
    
    # Calculate the biaxiality ratio
    biaxiality_ratio = sigma_2 / sigma_1
    
    return biaxiality_ratio

def calculate_von_mises_stress(S1, S2, S3=0):
    """
    Calculate the von Mises stress from the principal stresses.
    
    Parameters:
    S1 (float): First principal stress in MPa.
    S2 (float): Second principal stress in MPa.
    S3 (float): Third principal stress in MPa, assumed to be zero for plane stress condition.
    
    Returns:
    float: The von Mises stress in MPa.
    """
    # Calculate the von Mises stress using the principal stresses
    sigma_vm = np.sqrt(((S1 - S2)**2 + (S1 - S3)**2 + (S2 - S3)**2) / 2)
    return sigma_vm


-------

#### **Load Input Files**

>**SG_sample_out_data_sg_filtered_v2_long_test.csv**
* Specifies the input CSV data

    - First column should be time in seconds
    
    - Remaining columns are strains in each rosette channel, (units are in [μmm/mm])

>**rosette_angles.csv**
* Specifies the angles of each channel of a rosette

    - First column should be the channel name of rosette (SG1_1, SG2 etc.)

    - Remaining columns should be angles in degrees wrt to preferred axis

In [12]:
# Reading the files

# Dialog window for specifying the location of input file path for measured test data
# import sys
# from PyQt5.QtWidgets import QApplication, QFileDialog

# app = QApplication(sys.argv)
# file_path, _ = QFileDialog().getOpenFileName(None, 'Open File', '', 'All Files (*);;Text Files (*.txt)')

# if file_path:
#     print("Selected test data:", file_path)

# app = QApplication(sys.argv)
# angles_file_path, _ = QFileDialog().getOpenFileName(None, 'Open File', '', 'All Files (*);;Text Files (*.txt)')

# if angles_file_path:
#     print("Selected rosette angles data:", angles_file_path)


# Load the CSV file into a pandas DataFrame
file_path = 'SG_sample_out_data_sg_filtered_v2_long_test.csv'  # Path to the provided file
data = pd.read_csv(file_path)  # Read the entire data
print("Selected test data:", file_path)

# Load the CSV file containing the rosette angles
angles_file_path = 'rosette_angles.csv'
rosette_angles_df = pd.read_csv(angles_file_path)
print("Selected rosette angles data:", angles_file_path)

# Save 'Data No' and 'Time' columns for later
data_number = data['Data No']
time = data['Time']

# Filter only the strain gauge data, skipping the first two columns
strain_gauge_data = data.iloc[:, 2:].filter(regex='SG')

# Calculate the number of rosettes (each rosette has 3 strain measurements)
num_rosettes = (len(strain_gauge_data.columns)) // 3

Selected test data: SG_sample_out_data_sg_filtered_v2_long_test.csv
Selected rosette angles data: rosette_angles.csv


----
#### **Main Calculation Loop**

In [13]:
# Process each set of strain gauges for each rosette
for i in range(num_rosettes):
    
    rosette_row = rosette_angles_df[rosette_angles_df['SG'] == (i+1)]

    # If the rosette is found in the angles file, proceed with calculations
    if not rosette_row.empty:
        
        # Fetch the angles for the current rosette
        current_angles = rosette_row.iloc[0, 1:].values  # This will select all angle values in the row

        strain_gauge_cols = [f'SG{i+1}_1', f'SG{i+1}_2', f'SG{i+1}_3']
        strains = strain_gauge_data[strain_gauge_cols].values  # in microstrains

        # Transform strains to the global coordinate system using the current rosette's angles
        global_strains = np.array([transform_strains_to_global(*strain, current_angles) for strain in strains])

        principal_strains = np.array([calculate_principal_strains(strain[0], strain[1], strain[2]) for strain in global_strains])  # in microstrains
        principal_stresses = np.array([calculate_principal_stresses(strain, E, v) for strain in principal_strains])  # principal strains are already in strains
        principal_strain_orientation = np.array([calculate_principal_strain_orientation(strain[0], strain[1], strain[2]) for strain in global_strains])
        biaxiality_ratios = calculate_biaxiality_ratio(principal_stresses[:, 0], principal_stresses[:, 1])
        von_mises_stresses = np.array([calculate_von_mises_stress(*stress) for stress in principal_stresses])

        # Add the calculated data to the DataFrame using the correct naming convention
        strain_gauge_data[f'SG{i+1}_epsilon_x [με]'], strain_gauge_data[f'SG{i+1}_epsilon_y [με]'], strain_gauge_data[f'SG{i+1}_gamma_xy [με]'] = global_strains.T
        strain_gauge_data[f'SG{i+1}_sigma_1 [MPa]'], strain_gauge_data[f'SG{i+1}_sigma_2 [MPa]'] = principal_stresses.T
        strain_gauge_data[f'SG{i+1}_theta_p [°]'] = principal_strain_orientation
        strain_gauge_data[f'SG{i+1}_Biaxiality_Ratio'] = biaxiality_ratios
        strain_gauge_data[f'SG{i+1}_von_Mises [MPa]'] = von_mises_stresses

    else:
        print(f'Angles for Rosette {i+1} not found in angles file.')
        # Fill the DataFrame with NaNs for this rosette's calculations
        strain_gauge_data[f'SG{i+1}_epsilon_x [με]'], strain_gauge_data[f'SG{i+1}_epsilon_y [με]'], strain_gauge_data[f'SG{i+1}_gamma_xy [με]'] = global_strains.T
        strain_gauge_data[f'SG{i+1}_sigma_1 [MPa]'], strain_gauge_data[f'SG{i+1}_sigma_2 [MPa]'] = principal_stresses.T
        strain_gauge_data[f'SG{i+1}_theta_p [°]'] = principal_strain_orientation
        strain_gauge_data[f'SG{i+1}_Biaxiality_Ratio'] = biaxiality_ratios
        strain_gauge_data[f'SG{i+1}_von_Mises [MPa]'] = von_mises_stresses

# Add the 'Data No' and 'Time' columns back to the DataFrame
strain_gauge_data.insert(0, 'Time [s]', time)
#strain_gauge_data.insert(0, 'Data No', data_number)

# Display the DataFrame with the new calculated columns to verify
#print(strain_gauge_data.head())

excel_file_path = "strain_gauge_data_results.xlsx"
strain_gauge_data.to_excel(excel_file_path, index=False)

# Load the workbook and select the active worksheet
wb = load_workbook(excel_file_path)
ws = wb.active

# Determine the columns to hide
# Assuming 'Time' is in the first column and 'von Mises' stress columns are identified by a pattern
columns_to_hide = [col for col in ws.columns if col[0].value is not None 
                   and 'von_Mises' not in col[0].value 
                   and col[0].value != 'Time']

# Hide these columns
for column in columns_to_hide:
    ws.column_dimensions[column[0].column_letter].hidden = True

wb.save(excel_file_path)

strain_gauge_data

#endregion

,Time [s],SG1_1,SG1_2,SG1_3,SG2_1,SG2_2,SG2_3,SG1_epsilon_x [με],SG1_epsilon_y [με],SG1_gamma_xy [με],...,SG1_Biaxiality_Ratio,SG1_von_Mises [MPa],SG2_epsilon_x [με],SG2_epsilon_y [με],SG2_gamma_xy [με],SG2_sigma_1 [MPa],SG2_sigma_2 [MPa],SG2_theta_p [°],SG2_Biaxiality_Ratio,SG2_von_Mises [MPa]
0,0.0002,0.176820,0.079159,-0.039136,0.176820,0.079159,-0.039136,0.176820,-0.039136,0.020634,...,0.082006,0.034962,0.079159,0.058525,-0.215956,0.036357,0.002981,137.728943,0.082006,0.034962
1,0.0004,0.033286,0.079361,0.034511,0.033286,0.079361,0.034511,0.033286,0.034511,0.090925,...,0.161295,0.015511,0.079361,-0.011564,0.001225,0.016680,0.002690,0.385939,0.161295,0.015511
2,0.0006,-2.079800,-0.607560,0.676880,-2.079800,-0.607560,0.676880,-2.079800,0.676880,0.187800,...,-0.029365,0.419156,-0.607560,-0.795360,2.756680,0.012127,-0.412961,43.051361,-0.029365,0.419156
3,0.0008,-6.545800,-2.144700,1.997400,-6.545800,-2.144700,1.997400,-6.545800,1.997400,0.259000,...,-0.005890,1.311109,-2.144700,-2.403700,8.543200,0.007700,-1.307243,44.131762,-0.005890,1.311109
4,0.0010,-12.295000,-4.146900,3.711600,-12.295000,-4.146900,3.711600,-12.295000,3.711600,0.289600,...,-0.002148,2.460322,-4.146900,-4.436500,16.006600,0.005278,-2.457678,44.481744,-0.002148,2.460322
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1495,0.2992,71.656000,25.646000,-21.586000,71.656000,25.646000,-21.586000,71.656000,-21.586000,1.222000,...,-0.001411,14.336055,25.646000,24.424000,-93.242000,14.325935,-0.020220,135.375429,-0.001411,14.336055
1496,0.2994,52.097000,18.646000,-15.694000,52.097000,18.646000,-15.694000,52.097000,-15.694000,0.889000,...,-0.001413,10.422933,18.646000,17.757000,-67.791000,10.415569,-0.014712,135.375662,-0.001413,10.422933
1497,0.2996,32.297000,11.559000,-9.729000,32.297000,11.559000,-9.729000,32.297000,-9.729000,0.550000,...,-0.001401,6.461574,11.559000,11.009000,-42.026000,6.457046,-0.009046,135.374897,-0.001401,6.461574
1498,0.2998,12.348000,4.419000,-3.719000,12.348000,4.419000,-3.719000,12.348000,-3.719000,0.209000,...,-0.001342,2.470400,4.419000,4.210000,-16.067000,2.468742,-0.003313,135.372632,-0.001342,2.470400


##### Output Columns

In [14]:
# Checking the data inside the output excel
for column in ws.columns:
        print(f"Header: {column[0].value}, Column: {column[0].column_letter}")



Header: Time [s], Column: A
Header: SG1_1, Column: B
Header: SG1_2, Column: C
Header: SG1_3, Column: D
Header: SG2_1, Column: E
Header: SG2_2, Column: F
Header: SG2_3, Column: G
Header: SG1_epsilon_x [με], Column: H
Header: SG1_epsilon_y [με], Column: I
Header: SG1_gamma_xy [με], Column: J
Header: SG1_sigma_1 [MPa], Column: K
Header: SG1_sigma_2 [MPa], Column: L
Header: SG1_theta_p [°], Column: M
Header: SG1_Biaxiality_Ratio, Column: N
Header: SG1_von_Mises [MPa], Column: O
Header: SG2_epsilon_x [με], Column: P
Header: SG2_epsilon_y [με], Column: Q
Header: SG2_gamma_xy [με], Column: R
Header: SG2_sigma_1 [MPa], Column: S
Header: SG2_sigma_2 [MPa], Column: T
Header: SG2_theta_p [°], Column: U
Header: SG2_Biaxiality_Ratio, Column: V
Header: SG2_von_Mises [MPa], Column: W


---
#### **Comparison between real measurements & FEA**

In [15]:
# Plotting the data
import numpy as np
import plotly.graph_objs as go
from scipy.interpolate import interp1d
import ipywidgets as widgets
from IPython.display import display
# import endaq

# endaq.plot.utilities.set_theme(theme='endaq')

# Generate a pure sine wave as one dataset and offset it
x1 = np.linspace(0, 10, 100)
y1 = np.sin(x1) + 1.5

# Generate a slightly randomized sine wave as the other dataset and offset it
x2 = np.linspace(0, 10, 50)
y2 = np.sin(x2) + np.random.normal(0, 0.1, 50) + 1.5

# Create interpolation functions for both datasets
interp_func1 = interp1d(x1, y1, kind='linear', fill_value='extrapolate')
interp_func2 = interp1d(x2, y2, kind='linear', fill_value='extrapolate')

# Create a common set of x values for comparison
common_x = x1

# Calculate the interpolated y values for both datasets
interpolated_y1 = interp_func1(common_x)
interpolated_y2 = interp_func2(common_x)

# Calculate the relative error as a percentage
relative_error_percent = ((interpolated_y2 - interpolated_y1) / interpolated_y1) * 100

# Calculate the difference between the offset sine waves
difference = interpolated_y1 - interpolated_y2

# Initialize the plot as a FigureWidget with subplots
fig = go.FigureWidget(
    data=[
        go.Scatter(x=common_x, y=interpolated_y1, mode='lines', name='y1_Interpolated ', xaxis='x1', yaxis='y1'),
        go.Scatter(x=common_x, y=interpolated_y2, mode='lines', name='y2_Interpolated ', xaxis='x1', yaxis='y1'),
        go.Scatter(x=common_x, y=relative_error_percent, mode='lines', name='Error (%)', xaxis='x2', yaxis='y2'),
        go.Scatter(x=common_x, y=difference, mode='lines', name='Δy', xaxis='x3', yaxis='y3')
    ],
    layout=go.Layout(
        title="Comparison Between Datasets",
        height=800,  # Initial height
        xaxis=dict(domain=[0, 1], anchor='y1'),
        yaxis=dict(domain=[0.68, 1], title='y'),  # Adjust domain for spacing
        xaxis2=dict(domain=[0, 1], anchor='y2', matches='x1'),
        yaxis2=dict(domain=[0.36, 0.64], title='Error(%)'),  # Adjust domain for spacing
        xaxis3=dict(domain=[0, 1], anchor='y3', matches='x1'),
        yaxis3=dict(domain=[0, 0.32], title='Δy'),  # Adjust domain for spacing
        showlegend=True
    )
)

# Function to update the height of the plot
def update_height(height):
    fig.layout.height = height

# Create a slider widget for total_height
height_slider = widgets.IntSlider(value=500, min=300, max=1500, step=50, description='Total Height')

# Display the slider and the plot
display(height_slider)
display(fig)

# Add an observer to the slider to update the plot height
def on_height_change(change):
    update_height(change.new)

height_slider.observe(on_height_change, names='value')

#endregion

IntSlider(value=500, description='Total Height', max=1500, min=300, step=50)

FigureWidget({
    'data': [{'mode': 'lines',
              'name': 'y1_Interpolated ',
              'type': 'scatter',
              'uid': '6428dba7-1dc7-472d-a33d-96ba884aa89f',
              'x': array([ 0.        ,  0.1010101 ,  0.2020202 ,  0.3030303 ,  0.4040404 ,
                           0.50505051,  0.60606061,  0.70707071,  0.80808081,  0.90909091,
                           1.01010101,  1.11111111,  1.21212121,  1.31313131,  1.41414141,
                           1.51515152,  1.61616162,  1.71717172,  1.81818182,  1.91919192,
                           2.02020202,  2.12121212,  2.22222222,  2.32323232,  2.42424242,
                           2.52525253,  2.62626263,  2.72727273,  2.82828283,  2.92929293,
                           3.03030303,  3.13131313,  3.23232323,  3.33333333,  3.43434343,
                           3.53535354,  3.63636364,  3.73737374,  3.83838384,  3.93939394,
                           4.04040404,  4.14141414,  4.24242424,  4.34343434,  4.44444444,

In [16]:
file_path

'SG_sample_out_data_sg_filtered_v2_long_test.csv'